# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Login issues (SaaS)

This notebook contains Kusto queries that can help getting to the root cause of a login issue for an environment in the online version of Business Central (SaaS). Each section in the notebook contains links to the TSG part of the authorization telemetry documentation in [aka.ms/bctelemetry](aka.ms/bctelemetry), as well as Kusto queries that help dive into a specific area. 

NB! The signal used in this notebook is only available in version 16.2 (or newer) of Business Central online, so check the version of your environment.

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [27]:
# load the KQLmagic module
%reload_ext Kqlmagic

# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2. Define filters
This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName: 

In [28]:
aadTenantId = "<Add AAD tenant id here>"
environmentName = "<add environment name here>"

# Analyze the login flow
Now you can run Kusto queries to look for possible root causes for login issues.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Authentication

Authentication in the online version of Business Central happens strictly in Azure Active Directory (AAD). Only when a user is authenticated in AAD, a session is attempted to be created in the Business Central server (NST). When dealing with login issues, check for *absense* of signal in eventIds for Authorization in the pre-open company of opening a session in the NST (eventIds RT0001 and RT0003) to determine if the issue is related to AAD (e.g. user is disabled, wrong password, failed MFA)

Read more in the Security Guide here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

In [29]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId in ('RT0001', 'RT0003' )
| summarize request_count=count() by bin(timestamp, 1h) | render timechart title= 'Number of pre-open company authorization attempts in the last day'

[{'timestamp': datetime.datetime(2020, 9, 2, 16, 0, tzinfo=tzutc()), 'request_count': 9},
 {'timestamp': datetime.datetime(2020, 9, 2, 13, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 14, 0, tzinfo=tzutc()), 'request_count': 11},
 {'timestamp': datetime.datetime(2020, 9, 2, 15, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 9, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 10, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 11, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 12, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 4, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 6, 0, tzinfo=tzutc()), 'request_count': 9},
 {'timestamp': datetime.datetime(2020, 9, 2, 7, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 8, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 5, 0, tzinfo=tzutc()), 'request_count': 11},
 {'timestamp': datetime.datetime(2020, 9, 1, 21, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 1, 22, 0, tzinfo=tzutc()), 'request_count': 11},
 {'timestamp': datetime.datetime(2020, 9, 2, 3, 0, tzinfo=tzutc()), 'request_count': 9},
 {'timestamp': datetime.datetime(2020, 9, 2, 1, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 2, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 1, 19, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 1, 20, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 1, 18, 0, tzinfo=tzutc()), 'request_count': 8},
 {'timestamp': datetime.datetime(2020, 9, 1, 23, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 0, 0, tzinfo=tzutc()), 'request_count': 10},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 0, tzinfo=tzutc()), 'request_count': 1}]

## Authorization failures (pre-open company)

A user can fail authorization before the open company trigger is executed for a number of different reasons:
* The user was successfully authenticated in Azure Active Directory but the user account is disabled in Business Central.
* A user successfully authenticated in Azure Active Directory but the user does not have any entitlements in Business Central (license issue)

Read more about these types of failures in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorizationfailedpreopencompany

In [34]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0001'
| project timestamp
, guestUser = customDimensions.guestUser
, userType = customDimensions.userType
, failureReason = customDimensions.failureReason
, entitlementSetIds = customDimensions.entitlementSetIds
| order by timestamp desc
| limit 100

,timestamp,guestUser,userType,failureReason,entitlementSetIds


[]

## Authorization failures (in the open company process)

Events show up here for a number of different reasons
* The company name is invalid
* User has no permission to access the company
* The environment is locked
* The license has expired or the trial period has ended
* The user's license is not valid for use on production companies

Read more in the authorization signal docs here: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-authorization-trace#authorization-failed-open-company


In [35]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0002'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, failureReason = customDimensions.failureReason
| order by timestamp desc
| limit 100

,timestamp,clientType,companyName,failureReason


[]

## Successful logins (authentication in AAD succeded, authorization in the Business Central server succeeded)

If the user can authenticate against AAD and the two authorization steps inside the Business Central server succeeds, then a session is created and the user has successfully logged in.

Read more about application security in the Security Guide here:  https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/security/security-application#authentication

In [36]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where timestamp > ago(1d) // adjust accordingly to your analysis
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0004'
| project timestamp
, clientType = customDimensions.clientType
, companyName = customDimensions.companyName
, totalTimeInMS = toreal(totimespan(customDimensions.totalTime))/10000 // totalTime is measured in ticks, divide by 10000 to get milliseconds
| order by timestamp desc
| limit 100

,timestamp,clientType,companyName,totalTimeInMS
0,2020-09-02 18:14:46.756838900+00:00,Background,"CRONUS USA, Inc.",10.0171
1,2020-09-02 18:13:16.026500800+00:00,Background,"CRONUS USA, Inc.",5.9782
2,2020-09-02 18:11:44.258595100+00:00,Background,"CRONUS USA, Inc.",9.2422
3,2020-09-02 18:10:12.989594100+00:00,Background,"CRONUS USA, Inc.",11.9938
4,2020-09-02 18:08:42.445172900+00:00,Background,"CRONUS USA, Inc.",11.5650
5,2020-09-02 18:07:12.087972200+00:00,Background,"CRONUS USA, Inc.",17.2654
6,2020-09-02 18:05:40.766240700+00:00,Background,"CRONUS USA, Inc.",9.3166
7,2020-09-02 18:04:09.408706+00:00,Background,"CRONUS USA, Inc.",36.2553
8,2020-09-02 18:02:38.220527500+00:00,Background,"CRONUS USA, Inc.",6.9695
9,2020-09-02 18:01:07.891536+00:00,Background,"CRONUS USA, Inc.",10.9489


[{'timestamp': datetime.datetime(2020, 9, 2, 18, 14, 46, 756838, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.0171},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 13, 16, 26500, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.9782},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 11, 44, 258595, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 9.2422},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 10, 12, 989594, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 11.9938},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 8, 42, 445172, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 11.565},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 7, 12, 87972, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 17.2654},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 5, 40, 766240, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 9.3166},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 4, 9, 408706, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 36.2553},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 2, 38, 220527, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 6.9695},
 {'timestamp': datetime.datetime(2020, 9, 2, 18, 1, 7, 891536, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 10.9489},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 59, 36, 365000, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 6.776},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 58, 5, 521569, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 7.0006},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 56, 34, 740994, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.9161},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 55, 4, 418416, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 5.7305},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 53, 33, 279728, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 3.9506},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 52, 2, 820225, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0841},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 50, 32, 541658, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0121},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 49, 1, 976889, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0409},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 47, 30, 660180, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.5078},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 46, 0, 469446, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 3.9726},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 44, 29, 894954, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.0172},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 42, 59, 671278, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 7.9305},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 41, 24, 173459, tzinfo=tzutc()), 'clientType': 'Background', 'companyName': 'CRONUS USA, Inc.', 'totalTimeInMS': 4.3325},
 {'timestamp': datetime.datetime(2020, 9, 2, 17, 39, 53, 511556, tzinfo=tzutc(

In [40]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0004'
    and timestamp > ago(1d)
| extend clientType = tostring( customDimensions.clientType )
| summarize count=count() by clientType, bin(timestamp, 1h)
| render timechart title= 'Number of successful logins the last day (shown by client/session type)'

[{'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 18, 0, tzinfo=tzutc()), 'count': 15},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 17, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 16, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 13, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 14, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 15, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 4, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 5, 0, tzinfo=tzutc()), 'count': 33},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 6, 0, tzinfo=tzutc()), 'count': 36},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 7, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 8, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 9, 0, tzinfo=tzutc()), 'count': 42},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 10, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 11, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 12, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 0, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 1, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 2, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 1, 18, 0, tzinfo=tzutc()), 'count': 22},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 1, 19, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 1, 20, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 1, 21, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 1, 22, 0, tzinfo=tzutc()), 'count': 40},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 1, 23, 0, tzinfo=tzutc()), 'count': 39},
 {'clientType': 'Background', 'timestamp': datetime.datetime(2020, 9, 2, 3, 0, tzinfo=tzutc()), 'count': 39}]

In [39]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId in ('RT0001', 'RT0002', 'RT0004')
    and timestamp > ago(1d)
| extend attemptType = case(
    customDimensions.eventId == 'RT0001', 'Failure before open company' ,
    customDimensions.eventId == 'RT0002', 'Failure in open company trigger' ,
    customDimensions.eventId == 'RT0004', 'Successful login' , 
    'Unknown reason'
)
| summarize count=count() by attemptType, bin(timestamp, 1h)
| render timechart title= 'Number of login attempts the last day (shown by success/failure)'

[{'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 18, 0, tzinfo=tzutc()), 'count': 15},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 17, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 16, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 13, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 14, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 15, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 9, 0, tzinfo=tzutc()), 'count': 42},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 10, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 11, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 12, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 4, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 5, 0, tzinfo=tzutc()), 'count': 33},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 6, 0, tzinfo=tzutc()), 'count': 36},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 7, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 8, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 3, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 1, 21, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 1, 22, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 1, 23, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 1, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 1, 18, 0, tzinfo=tzutc()), 'count': 23},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 1, 19, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 1, 20, 0, tzinfo=tzutc()), 'count': 39},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 2, 0, tzinfo=tzutc()), 'count': 40},
 {'attemptType': 'Successful login', 'timestamp': datetime.datetime(2020, 9, 2, 0, 0, tzinfo=tzutc()), 'count': 40}]